Preparing to submit wold stranded samples....

This got a little messy because a couple libraries needed a top up.

In [1]:
import os
import sys
import requests
import pandas
import paramiko
import json
from IPython import display
from pathlib import Path
import configparser

In [2]:
EC = str(Path('~/proj/encoded_client').expanduser())
if EC not in sys.path:
    sys.path.append(EC)

from encoded_client.encoded import ENCODED, Document, typed_column_parser, DCCValidator

In [3]:
HTSW = str(Path('~/proj/htsworkflow').expanduser())
if HTSW not in sys.path:
    sys.path.append(HTSW)

In [4]:

from htsworkflow.submission.aws_submission import run_aws_cp
from htsworkflow.util.api import (
    add_auth_options,
    make_auth_from_opts,
    HtswApi,
)

In [5]:
config = configparser.ConfigParser()
config.read([os.path.expanduser('~/.htsworkflow.ini'),
             '/etc/htsworkflow.ini'
             ])

SECTION = 'sequence_archive'
if config.has_section(SECTION):
    apiid = config.get(SECTION, 'apiid')
    apikey = config.get(SECTION, 'apikey')
    apihost = config.get(SECTION, 'host')

apihost='http://jumpgate.caltech.edu'    
auth = {'apiid': apiid, 'apikey': apikey }
htsw = HtswApi(apihost, auth)

In [6]:
# live server & control file
server = ENCODED('www.encodeproject.org')
spreadsheet_name = Path('~/woldlab/ENCODE/stranded-23239-23266-left-cortex.xlsx').expanduser()
engine=None
#engine='odf'

# test server & datafile
#server = ENCODED('test.encodedcc.org')
#spreadsheet_name = os.path.expanduser('~diane/woldlab/ENCODE/C1-encode3-limb-2017-testserver.ods')

server.load_netrc()
validator = DCCValidator(server)

In [7]:
award = 'UM1HG009443'

# Lookup biosample ontologies

Lookup any biosample ontologies that are already present

In [8]:
biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)

for i, row in biosample.iterrows():
    if not pandas.isnull(row.accession) and row.accession.startswith('E'):
        obj = server.get_json(row.accession)
        biosample_ontology = obj['biosample_ontology']
        if isinstance(biosample_ontology, dict):
            biosample.loc[i, 'biosample_ontology'] = biosample_ontology['@id']
            biosample.loc[i, 'biosample_term_name:skip'] = biosample_ontology['term_name']
        biosample.loc[i, 'source'] = obj['source']['@id']
            
biosample

,uuid,accession,aliases:array,biosample_ontology,biosample_term_name:skip,description,library_id:skip,cDNA_sample:skip,organism,source,lab,award
0,NaN,ENCBS398MXZ,barbara-wold:ENC4_cDNA521,/biosample-types/tissue_UBERON_0000956/,cerebral cortex,B6Cast F1 PND4 Male 03 L. cortex,23239,ENC4_cDNA521,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443
1,NaN,ENCBS178USG,barbara-wold:ENC4_cDNA522,/biosample-types/tissue_UBERON_0000956/,cerebral cortex,B6Cast F1 PND4 Male 04 L. cortex,23240,ENC4_cDNA522,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443
2,NaN,ENCBS151XIY,barbara-wold:ENC4_cDNA523,/biosample-types/tissue_UBERON_0000956/,cerebral cortex,B6Cast F1 PND4 Female 03 L. cortex,23241,ENC4_cDNA523,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443
3,NaN,ENCBS567ZSA,barbara-wold:ENC4_cDNA524,/biosample-types/tissue_UBERON_0000956/,cerebral cortex,B6Cast F1 PND4 Female 04 L. cortex,23242,ENC4_cDNA524,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443
4,NaN,ENCBS941MMJ,barbara-wold:ENC4_cDNA525,/biosample-types/tissue_UBERON_0000956/,cerebral cortex,B6Cast F1 PND10 Male 03 L. cortex,23243,ENC4_cDNA525,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443
5,NaN,ENCBS919NNU,barbara-wold:ENC4_cDNA526,/biosample-types/tissue_UBERON_0000956/,cerebral cortex,B6Cast F1 PND10 Male 04 L. cortex,23244,ENC4_cDNA526,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443
6,NaN,ENCBS056HYO,barbara-wold:ENC4_cDNA527,/biosample-types/tissue_UBERON_0000956/,cerebral cortex,B6Cast F1 PND10 Female 03 L. cortex,23245,ENC4_cDNA527,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443
7,NaN,ENCBS716XOP,barbara-wold:ENC4_cDNA528,/biosample-types/tissue_UBERON_0000956/,cerebral cortex,B6Cast F1 PND10 Female 04 L. cortex,23246,ENC4_cDNA528,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443
8,NaN,ENCBS993UDC,barbara-wold:ENC4_cDNA529,/biosample-types/tissue_UBERON_0000956/,cerebral cortex,B6Cast F1 PND14 3x Males 03 L. cortex,23247,ENC4_cDNA529,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443
9,NaN,ENCBS233WUB,barbara-wold:ENC4_cDNA530,/biosample-types/tissue_UBERON_0000956/,cerebral cortex,B6Cast F1 PND14 3x Males 04 L. cortex,23248,ENC4_cDNA530,/organisms/mouse/,/sources/jackson-labs/,barbara-wold,UM1HG009443


In [9]:
#biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

# Register Biosamples

In [10]:
biosamples = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)
created = server.post_sheet('/biosamples/', biosamples, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

0


In [11]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [12]:
if created:
    biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

In [31]:
print(server.server)
biosamples = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)
for i, row in biosamples.iterrows():
    biosample_id = '/biosample/{}/'.format(row.accession)
    biosample = server.get_json(biosample_id)
    if row['biosample_ontology'] != biosample['biosample_ontology']['@id']:
        print(row.accession, row['biosample_ontology'], biosample['biosample_ontology']['@id'])
        #print(server.patch_json(biosample_id, {"biosample_ontology": row['biosample_ontology']}))

www.encodeproject.org


# Update biosample aliases

In [13]:
biosamples = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)
for i, row in biosamples.iterrows():
    obj = server.get_json('/biosamples/{}/'.format(row.accession))
    new_aliases = obj['aliases'].copy()
    sheet_aliases = typed_column_parser('aliases:array', row['aliases:array'])[1]
    for alias in sheet_aliases:
        if alias not in obj['aliases']:
            new_aliases.append(alias)
    if len(new_aliases) > len(sheet_aliases):
        print("Would update: {} {} -> {}".format(row.accession, row['aliases:array'], new_aliases))

In [14]:
new_aliases, sheet_aliases

(['barbara-wold:ENC4_cDNA548'], ['barbara-wold:ENC4_cDNA548'])

# Register Libraries

In [15]:
print(spreadsheet_name)
libraries = pandas.read_excel(spreadsheet_name, sheet_name='Library', header=0, engine=engine)
for i, row in libraries.iterrows():
    accession = row['accession']
    alias = row['aliases:array']
    fragment = row['average_fragment_size:integer']
    library_id = alias[len('barbara-wold:'):]
    jumpgate_info = htsw.get_library(library_id)
    if not pandas.isnull(accession):
        dcc_info = server.get_json(accession)
        print(library_id, fragment, jumpgate_info['insert_size'], dcc_info.get('average_fragment_size'))
        #if jumpgate_info['insert_size'] != dcc_info.get('average_fragment_size'):
        #    server.patch_json(dcc_info['@id'], {'average_fragment_size': int(jumpgate_info['insert_size'])})

/home/diane/woldlab/ENCODE/stranded-23239-23266-left-cortex.xlsx
23239 348 248 348
23240 343 243 343
23241 352 252 352
23242 351 251 351
23243 338 238 338
23244 349 249 349
23245 349 249 349
23246 347 247 347
23247 353 253 353
23248 340 240 340
23249 342 242 342
23250 348 248 348
23251 346 246 346
23252 341 241 341
23253 378 278 378
23254 342 242 342
23255 345 245 345
23256 341 241 341
23257 337 237 337
23258 332 232 332
23259 344 244 344
23260 330 230 330
23261 341 241 341
23262 343 243 343
23263 347 247 347
23264 345 245 345
23265 340 240 340
23266 340 240 340


In [16]:
print(spreadsheet_name)
libraries = pandas.read_excel(spreadsheet_name, sheet_name='Library', header=0, engine=engine)
created = server.post_sheet('/libraries/', 
                            libraries,
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

/home/diane/woldlab/ENCODE/stranded-23239-23266-left-cortex.xlsx
0


In [17]:
if created:
    libraries.to_excel('/dev/shm/libraries.xlsx', index=False)

# Register Experiments

In [18]:
print(server.server)
experiments = pandas.read_excel(spreadsheet_name, sheet_name='Experiment', header=0, engine=engine)
experiments = experiments[experiments['accession'] != 'barbara approval needed']
created = server.post_sheet('/experiments/', 
                            experiments, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

www.encodeproject.org
0


In [19]:
if created:
    experiments.to_excel('/dev/shm/experiments.xlsx', index=False)

# Validate Experiment Ontologies

In [27]:
print(server.server)
experiments = pandas.read_excel(spreadsheet_name, sheet_name='Experiment', header=0, engine=engine)
for i, row in experiments.iterrows():
    experiment_id = '/experiments/{}/'.format(row.accession)
    experiment = server.get_json(experiment_id)
    if row['biosample_ontology'] != experiment['biosample_ontology']['@id']:
        print(row.accession, row['biosample_ontology'], experiment['biosample_ontology']['@id'])
        #print(server.patch_json(experiment_id, {"biosample_ontology": row['biosample_ontology']}))

www.encodeproject.org


# Register Replicates

In [20]:
print(server.server)
print(spreadsheet_name)
replicates = pandas.read_excel(spreadsheet_name, sheet_name='Replicate', header=0, engine=engine)
replicates = replicates[replicates['uuid'] != 'barbara approval needed']
created = server.post_sheet('/replicates/',
                            replicates, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

www.encodeproject.org
/home/diane/woldlab/ENCODE/stranded-23239-23266-left-cortex.xlsx
0


In [21]:
if created:
    replicates.to_excel('/dev/shm/replicates.xlsx', index=False)

# Check Files

In [22]:
files = pandas.read_excel(spreadsheet_name, sheet_name='File', header=0, engine=engine)
created = server.post_sheet('/files/', files, verbose=True, dry_run=True, validator=validator)
print(len(created))

0
